# 環境設定

## 安裝 GPU 驅動相關套件

In [ ]:
# 更新 Linux 內的套件清單至最新版
!apt-get update

# 安裝 PCI 匯流排工具（PCI Utility）與 lshw (LiSt HardWare)，以便能偵測到 GPU
# -y：遇到詢問是否安裝，一律自動回答 yes
!apt-get install -y pciutils lshw

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
# 用 nVidia 的 System Management Interface (SMI) 確認 GPU 的確抓得到
!nvidia-smi

Fri Dec 20 05:49:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 安裝 Ollama

In [ ]:
# 至 https://ollama.com/download/linux
# 直接將安裝 Ollama 於 Linux 的指令貼上
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# 啟動 Ollama，讓它執行於背景中
# ollama serve: 用 Server 模式、而非互動模式執行 Ollama
# > server.log：將本應顯示於螢幕的訊息，轉向輸出至 server.log 這個檔備查
# 2>&1：2 為 stderr。將所有錯誤訊息，轉向 &1 (stdout，螢幕) 輸出。
# &：將程式啟動之後，馬上返回，不要等該程式執行完成
!ollama serve > server.log 2>&1 &

In [ ]:
# 將 Llama-3 模型下載，並做為此次的大語言模型
# ollama run <模型名稱>：下載並執行特定 LLM
# > model.log：將本應顯示於螢幕的訊息，轉向輸出至 model.log 這個檔備查
!ollama run gemma:7b > model.log 2>&1 &

# 注意：上述兩指令皆以「&」後綴，告知 Colab「不用等 Linux 執行完」。
# 但事實上，不論啟動為 Server，或下載 LLM，皆須 1~5 分鐘不等的時間。
# 可以查看 server.log、model.log 兩檔案內容，得知當前執行狀況。

# 將 nomic-embed-text 模型，拉下來備用
# nomic-embed-text 是一款能將文字向量化的嵌入層模型
!ollama pull nomic-embed-text > embedding.log 2>&1 &


## 安裝 LangChain + 連上大語言模型

In [ ]:
# 下載 LangChain，一套專門連上各種大語言模型的 Python 套件
!pip install langchain  # LangChain 核心元件
!pip install langchain-community  # 各種開源大語言模型連接函數套件

In [ ]:
# 載入 Ollama 以便連上後端 LLM
from langchain_community.llms import Ollama

# LLM 名稱需與 ollama run 後方名稱相同
llm = Ollama(model="gemma:7b")

<ipython-input-8-ee38639c5b59>:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:7b")


# RAG 環境設定及資料處理

## 引入向量資料庫

In [ ]:
# ChromaDB 是一款蠻受歡迎的開源式向量資料庫
!pip install chromadb

## 匯入課程資訊相關文件文件


In [ ]:
import os

In [ ]:
# 此函數用於下載google drive上的檔案
def doc_download(Dataset_File,Data_url):
    """
    雲端檔案讀入
    """
    if not os.path.isfile(Dataset_File):
      url = url_trans(Data_url)
      os.system(f"wget {url} -O {Dataset_File}")

In [ ]:
def url_trans(url):
    """
    將 Google Drive 網址轉換
    """
    if "https://drive.google.com/file/d/" in url:
        # 提取 ID 部分
        file_id = url.split("/file/d/")[1].split("/")[0]
        # 拼接成新的下載鏈接
        direct_url = f"https://drive.google.com/uc?id={file_id}"
        return direct_url
    else:
        raise ValueError("提供的網址格式不正確。請確認是 Google Drive 分享網址。")

In [ ]:
doc_download("content.sorted.csv","https://drive.google.com/file/d/1_24DZraBJ4bJUpCV3bLGSCXtKptla_LQ/view?usp=drive_link")

## 資料解析

In [ ]:
#載入 CSV 資料
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='content.sorted.csv')
data = loader.load()

### 若檔案非CSV檔時的處理流程

In [ ]:
# # 安裝 JSON 檔內容解析套件
# !pip install jq
# from pathlib import Path  # 匯入 pathlib 模組
# import json  # 匯入 json 模組
# from langchain_community.document_loaders import JSONLoader


In [ ]:
# #json檔解析
# def json_loader(file):
#   """
#     輸入檔案名稱解析檔案
#   """
#   file_path = Path(file)
#   with file_path.open(encoding="utf-8-sig") as f:
#       json_data = json.load(f)

#   return json_data

In [ ]:
# file_path = Path("test.json")
# with file_path.open() as f:
#   json_data = json.load(f)

# print(json_data)

In [ ]:
# !pip install pypdf

In [ ]:
# # 載入 PDF 資料
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader(test_pdf)
# test_doc = loader.load()

# print(test_doc)

In [ ]:
# # 載入網頁資料
# from langchain_community.document_loaders import WebBaseLoader

# url = "https://goinglearn.com.tw/about-this-site"
# loader = WebBaseLoader(url)
# web_docs = loader.load()

# print(web_docs)

## 資料合併

此專案只用到一份檔案故資料合併非必要操作

In [ ]:
all_docs = []

In [ ]:
def add_file(file):
  global all_docs
  all_docs += file
  print(all_docs)

In [ ]:
add_file(data)

[Document(metadata={'source': 'content.sorted.csv', 'row': 0}, page_content='course name: 民法總則\nsemester: 112-1\nteacher: 牛曰正\nreflection: 老師之後應該也不會在政大任教了（那就簡略寫寫），上課內容不難，主要是以講義所列課程大綱作說明、補充，課後要填回饋紙條蠻有趣的而且期末可以加分（事實上證明真的很需要😍）。期中是非常簡單的選擇題，期末就是殺人不眨眼的傢伙，選擇加每題都塞了爆多爭點的申論題，我也不知道我在寫啥但學期調分就是我的神。\nrating: 甜度：🍬🍬🍬🍬🍬 涼度：🧊🧊🧊🧊 分數：90-94 推薦指數：🍎🍎\n: '), Document(metadata={'source': 'content.sorted.csv', 'row': 1}, page_content='course name: 刑法（一）\nsemester: 112-1\nteacher: 謝如媛\nreflection: 非常喜歡如媛小淑女，上課節奏適中、教學內容扎實，考試內容不難（選擇題＋申論題），不過要注意的是學期成績是期中40分＋期末60分，分數扣起來很疼，如果像我一樣期末考爆就尷尬了。 另外要大大安利這門課的助教威齊，非常用心的編制講義與教學，在一開始進來法律系啥都不懂時真的都是透過威齊學習如何複習、如何做筆記、如何審題列架構，若還有不懂也可以寄信問他，他都會很有耐心很認真的把你教到會，是我非常仰慕且尊敬的助教大人。希望學弟妹有福氣遇到人家，遇到也要把人家好好拱著，這麼好的助教真的很難遇到喔！\nrating: 甜度：🍬🍬🍬🍬 涼度：🧊🧊 分數：95-99 推薦指數：🍎🍎🍎🍎\n: '), Document(metadata={'source': 'content.sorted.csv', 'row': 2}, page_content='course name: 刑法（二）\nsemester: 112-2\nteacher: 謝如媛\nreflection: 非常喜歡如媛小淑女，上課節奏適中、教學內容扎實，考試內容不難（選擇題＋申論題），不過要注意的是學期成績是期中40分＋期末60分，分數扣起來很疼，如果像我一樣期末考爆就尷尬了。 另外要大

## 資料切割

In [ ]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
nomic-embed-text:latest    0a109f422b47    274 MB    21 seconds ago    
gemma:7b                   a72c7f4d0a15    5.0 GB    3 minutes ago     


In [ ]:
!ollama start nomic-embed-text

Error: accepts 0 arg(s), received 1


In [ ]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# 假設 all_docs 中的元素有些是字典，有些是 Document 類型的對象
all_docs_list = []
for doc in all_docs:
    if isinstance(doc, dict):  # 如果是字典，則轉換為 Document
        page_content = doc.get("reflection", "")  # 預設值為空字串
        metadata = {
            "課程名稱": doc.get("課程名稱", "未知課程"),
            "teacher": doc.get("teacher", "未知老師")  # 確保有提取teacher
        }  # 確保 metadata 是字典
        all_docs_list.append(Document(page_content=page_content, metadata=metadata))
    elif isinstance(doc, Document):  # 如果已經是 Document 類型，則直接使用
        page_content = doc.page_content

        # 使用正則表達式更新 metadata
        course_name_match = re.search(r'course name:\s*(.*?)\n', page_content)
        teacher_match = re.search(r'teacher:\s*(.*?)\n', page_content)

        # 如果找到了相應的匹配項，提取資料
        if course_name_match and teacher_match:
            course_name = course_name_match.group(1)
            teacher = teacher_match.group(1)

            # 更新原有 metadata
            doc.metadata.update({
                'course_name': course_name,
                'teacher': teacher
            })

        # 確保直接使用原始 Document 實例，無需重新創建
        all_docs_list.append(doc)
    else:
        raise ValueError(f"未知的資料格式：{type(doc)}")

# 初始化文本切分器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 切分文檔
splits = text_splitter.split_documents(all_docs_list)

# 列印切分後的文檔
for split in splits:
    print(split)



page_content='course name: 民法總則
semester: 112-1
teacher: 牛曰正
reflection: 老師之後應該也不會在政大任教了（那就簡略寫寫），上課內容不難，主要是以講義所列課程大綱作說明、補充，課後要填回饋紙條蠻有趣的而且期末可以加分（事實上證明真的很需要😍）。期中是非常簡單的選擇題，期末就是殺人不眨眼的傢伙，選擇加每題都塞了爆多爭點的申論題，我也不知道我在寫啥但學期調分就是我的神。
rating: 甜度：🍬🍬🍬🍬🍬 涼度：🧊🧊🧊🧊 分數：90-94 推薦指數：🍎🍎
:' metadata={'source': 'content.sorted.csv', 'row': 0, 'course_name': '民法總則', 'teacher': '牛曰正'}
page_content='course name: 刑法（一）
semester: 112-1
teacher: 謝如媛
reflection: 非常喜歡如媛小淑女，上課節奏適中、教學內容扎實，考試內容不難（選擇題＋申論題），不過要注意的是學期成績是期中40分＋期末60分，分數扣起來很疼，如果像我一樣期末考爆就尷尬了。 另外要大大安利這門課的助教威齊，非常用心的編制講義與教學，在一開始進來法律系啥都不懂時真的都是透過威齊學習如何複習、如何做筆記、如何審題列架構，若還有不懂也可以寄信問他，他都會很有耐心很認真的把你教到會，是我非常仰慕且尊敬的助教大人。希望學弟妹有福氣遇到人家，遇到也要把人家好好拱著，這麼好的助教真的很難遇到喔！
rating: 甜度：🍬🍬🍬🍬 涼度：🧊🧊 分數：95-99 推薦指數：🍎🍎🍎🍎
:' metadata={'source': 'content.sorted.csv', 'row': 1, 'course_name': '刑法（一）', 'teacher': '謝如媛'}
page_content='course name: 刑法（二）
semester: 112-2
teacher: 謝如媛
reflection: 非常喜歡如媛小淑女，上課節奏適中、教學內容扎實，考試內容不難（選擇題＋申論題），不過要注意的是學期成績是期中40分＋期末60分，分數扣起來很疼，如果像我一樣期末考爆就尷尬了。 另外要大大安利這門課的助教威齊，非常用心的編制

In [ ]:
# ... (previous code) ...

# Ensure the model is downloaded and started
!ollama start nomic-embed-text

#Check to see if it is downloaded
!ollama list

# 利用 nomic-embed-text 模型，執行文本向量化
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_db = Chroma.from_documents(documents=splits, embedding=embeddings)


Error: accepts 0 arg(s), received 1
NAME                       ID              SIZE      MODIFIED       
nomic-embed-text:latest    0a109f422b47    274 MB    36 seconds ago    
gemma:7b                   a72c7f4d0a15    5.0 GB    4 minutes ago     


<ipython-input-35-7fdae672ddb0>:13: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


# 提問及抽取資料

In [ ]:
!ollama pull gemma:7b


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ef311de6af9d... 100% ▕▏ 5.0 GB                         
pulling 097a36493f71... 100% ▕▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕▏  136 B                         
pulling 65bb16cf5983... 100% ▕▏  109 B                         
pulling 0c2a5137eb3c... 100% ▕▏  483 B                         
verifying sha256 digest 
writing manifest 
success 


## 問題理解

In [ ]:
rules = """
範例：
輸入：「民法總則的課程評價為何？」
回答：民法總則
輸入：「民總的課程評價怎麼樣呢？」
回答：民法總則
輸入：「程式設計的評價？」
回答：程式設計
輸入：尋找統計學相關課程評價
回答：統計學

根據上述範例回答下列敘述的課程名稱，切記勿直接回答輸入的問題，回答只會有課程名稱本身，不會有推理過程等額外資訊
"""

In [ ]:
# 讓模型理解使用者問題，並抽取課名的關鍵字以供RAG查詢
def input_trans(question):
    # 製作提問用的提詞（Prompt）
    formatted_prompt = rules + question
    # 丟入 LLM，取得答案
    msg = llm.invoke(formatted_prompt)
    return msg

## 根據抓取資料整理內容

In [ ]:
# 讓模型依據"未經整理抓取資料"和"整理後結果"的配對示例，去整理各種從RAG資料庫裡提取的資訊
def info_arrange(course_info):
  course_prompt = formatted_course + "根據上面整理輸入輸出轉換的範例以老師為主提整理下方課程資訊，切忌回答勿直接使用範例輸入輸出，且只整理跟下列課程相關的資訊，用中文而非英文回答:" + course_info
  # 丟入 LLM，取得答案
  msg = llm.invoke(course_prompt)
  return msg


In [ ]:
formatted_course = """
輸入範例1:
course name: 商事法
semester: 112-1
teacher: 王正偉
reflection: 課堂不點名，期中考試與期末報告為主要評分依據。對於法律零基礎的學生，教授會耐心指導並幫助學生通過課程。
rating: 甜度：4/5 涼度：4/5 分數：78
:

course name: 商事法總論與公司法＋公司法案例研習（二）
semester: 112-1
teacher: 周振鋒
reflection: 每週五上四個小時真的是要我命。不過老師常常安排專家學者在第四節講座，有稍微緩解不自覺靈魂出竅的狀況（對不擠我就是顆草莓）。考試可以帶大抄，這時候可以見識到大家word功力如何。建議考試前一週開始吃葉黃素，我當初沒吃，考完出來體感近視增加50度。曾經考慮要問老師可否帶放大鏡進考場。（當然如果你有讀熟才去考試、不用一直看大抄就不會遇到這種狀況）
rating: N/A
:

course name: 商事法
semester: 112-1
teacher: 鄭婷嫻
reflection: 老師教得很用心，認真聽課能學到不少法律知識。作業部分是小組合作，期末考試題目大多來自作業，有加分機會。
rating: 4/5 & 4/5
:

course name: 商事法
semester: N/A
teacher: 劉承愚
reflection: 上這堂課壓力有點大，要課前預習，但老師上課很精彩，會講到很多實務的東西，蠻有趣的。考試單選複選混在一起（就是你不知道這題是單選還是複選），但我也不知道我考試到底考幾分，分數有點玄
rating: 甜：🍭🍭🍭🍭🍭 涼：🧊🧊 分數：95-100
:

course name: 商事法
semester: 110-1
teacher: 陳盈如
reflection: 老師是一位非常漂亮的老師，講解法條非常詳細，會搭配案例來幫助理解。作業通常是小組形式，並且會有加分作業，期末考可以帶大抄。整體來說，這是一堂非常紮實且不太難的課程。
rating: 4 & 3

輸出範例1:
周振鋒
課程名稱：商事法總論與公司法＋公司法案例研習（二）

學期：112-1

課程特色：

每週安排4小時課程（含專家學者講座）。
考試可帶大抄，需製作詳細筆記。
學生評價：

長課時挑戰較大，適合有耐心與充分準備的學生。
整體課程重心放在實務案例與考試應對技巧。
鄭婷嫻
課程名稱：商事法

學期：112-1

課程特色：

作業以小組合作為主，期末考試題目多來自作業。
提供加分機會，期末考試強調知識運用與應用。
學生評價：

老師教學認真，適合用心學習的學生。
分數整體甜度與涼度相當高（4/5）。
劉承愚
課程名稱：商事法

學期：N/A

課程特色：

課前需要預習，上課內容精彩，重點放在實務應用。
考試包含單選與複選混合題型，需仔細準備。
學生評價：

老師講課有趣，但壓力偏大。
成績高甜（甜度🍭🍭🍭🍭🍭，涼度🧊🧊，分數約95-100分）。
陳盈如
課程名稱：商事法

學期：110-1

課程特色：

老師講解法條詳細，搭配案例幫助理解。
作業與期末考試設計較友善，期末考試可帶大抄。
學生評價：

老師非常漂亮，課程紮實且不難。
整體甜度與涼度適中（4/5 & 3/5），適合希望穩紮穩打學習法律基礎的學生。

輸入範例2:
course name: 中國大陸概論
semester: 110-2
teacher: 王信賢
reflection: 評分方式：50%課堂表現（主要是出席、小組報告的組內與組間互評）、50%期末考 分數：95-99 心得：這堂也是我超喜歡的通識課之一，老師上課非常有料，對中國的各方面都會有所介紹，同時也會分享一些老師去做田野的趣事！最後課程也會有兵推～整體負擔也不會太大，每次上正課都很希望老師可以繼續講下去，在期末考前主要只需要準備小組的口頭報告即可，助教也會事前和大家約時間給予大家建議，期末考的部分老師上課也會提示，有好好上課應該不會太難，不過老師不會提供PPT，只會提供講綱給大家做筆記
rating: 甜度：4.5/5 涼度：3/5
:

course name: 中國大陸概論
semester: 111-1
teacher: 林義鈞
reflection: 不開放加簽！是一門初選排一還不一定會上的搶手課程。 這個學期要寫五篇預習心得、三篇加分用的觀影心得和三篇加分用的講座心得，以核通而言這個作業量應該還算適中。第二週就會分組，討論課（18:00-19:00）和期末報告都會需要你的組員。 課程內容會介紹政治、社會、經濟、外交。前兩個小時是教授講述內容，課前會把PPT上傳到Moodle上給大家參看，有需要的人可以下載下來做筆記。教授為人相當親切、樂於稱讚學生，也會冷不防逗大家笑。我認為修過這堂課之後，對於兩岸的情勢有更全面性的理解，課後在聽別人討論相關議題的時候也會比較快融入；比較可惜的是因為時間因素，有些主題沒辦法討論太久。學期中的三次演講邀請了學界的專家來分享研究成果和所見所聞，我覺得很值得一聽。 最後一個小時是討論課，學期初將十幾組分成兩個班，各自讓兩位助教帶領討論對岸的時事議題。 我當時對這個領域很有興趣，所以上得很開心xDDD 但無論如何都是一堂值得推推的好課
rating: 甜度：4 涼度：3.8
:

course name: 中國大陸概論
semester: 1121
teacher: 林義鈞
reflection: 上課內容很豐富，基本上政治經濟社會都有講到，老師上課也很認真，可以學到很多東西，助教課氛圍也還算輕鬆，另外有指定教科書。作業有數次心得跟小組書面報告。心得就是閱讀完指定教科書特定章節的心得，書面報告也不會太難做。有期中期末，但期末可以帶參考資料(至少當時可以)。然後會有看電影跟演講，有寫心得可以加分。
rating: 甜度：5/5 涼度：4.5/5 分數：98
:

course name: 中國大陸概論
semester: N/A
teacher: 唐玉禮
reflection: 好像不太加簽。又是一堂大家都在划手機看電腦做自己事的課，老師其實人滿好的很親切，上課內容偏無聊，每個禮拜看老師給的報導寫600字心得，我自己覺得負擔不太重，期末小組上台報告外加個人報告(網路繳交即可)。偶爾點名，不雷但也不是大家會狂推的課，可以選。
rating: 甜度：4/5 涼度：3/5
:

course name: 中國大陸概論
semester: 107
teacher: 林義鈞
reflection: 沒有想像中甜，不知道有沒有故意壓低成績，但期中真的佛到爆，完全不用準備就可以100，期末申論題。助教人也很好，很認真在帶討論。
rating: 甜度：4/5 涼度：3/5
:

course name: 中國大陸概論
semester: 110-1
teacher: 林義鈞
reflection: 上課方式多元，包括授課、講師演講與電影欣賞。期中期末考難度適中，討論課與專題報告負擔適中，是負擔較輕的核心通識課，推薦對中國議題有興趣的同學選修。
rating: 甜：4/5 涼：4.5/5

輸出範例2:
**王信賢**

課程名稱：中國大陸概論

學期：110-2

課程特色：講課富有內容，教授有深度研究中國各方面，並分享個人經驗。期末考僅需口頭報告。

**林義鈞**

課程名稱：中國大陸概論

學期：111-1、112-1、107、110-1

課程特色：教授熱情洋溢，講課認真，並會引導討論。作業量適中，並有定期演講和電影欣賞。

**唐玉禮**

課程名稱：中國大陸概論

學期：N/A

課程特色：教授人滿有禮，但課程內容平淡。作業量輕微，但定期點名。

"""

## RAG資料抓取及回答整體流程

In [ ]:
# 定義向量資料庫抽取物件
retriever = vector_db.as_retriever(search_kwargs={"k": 200 })

# 根據與使用者問題的相似度，抽取相關資料(測試用，用於檢查抽取資料及模型理解問題的效果)
def get_rag_docs_test(question):
    # 依照與問題的相似度，抽取相關文本
    retrieved_docs = retriever.invoke(question)
    refined_docs = []
    # 將所有相關文本，以空行 \n\n 黏成一個大字串
    class_name = input_trans(question)
    print(class_name)
    for doc in retrieved_docs:
      if class_name in doc.metadata["course_name"]:  # 根據具體需求進行進一步篩選
        refined_docs.append(doc)
    relavent_contents = "\n\n".join(doc.page_content for doc in refined_docs)
    print("參考資料：" + "\n\n")
    output = info_arrange(relavent_contents)
    return output
    # return llm_ask(question, relavent_contents)



In [ ]:
# 定義向量資料庫抽取物件
retriever = vector_db.as_retriever(search_kwargs={"k": 200 })

# 根據與使用者問題的相似度，抽取相關資料
def get_rag_docs(question):
    # 依照與問題的相似度，抽取相關文本
    retrieved_docs = retriever.invoke(question)
    refined_docs = []
    # 將所有相關文本，以空行 \n\n 黏成一個大字串
    class_name = input_trans(question)
    for doc in retrieved_docs:
      if class_name in doc.metadata["course_name"]:  # 根據具體需求進行進一步篩選
        refined_docs.append(doc)
    relavent_contents = "\n\n".join(doc.page_content for doc in refined_docs)
    # 將問題 + RAG 相關文本，一口氣丟入 LLM，取得答案
    output = info_arrange(relavent_contents)
    return output
    # return llm_ask(question, relavent_contents)



In [ ]:
# 定義一個能丟入「使用者問題」與「RAG 抽取之相關文本」，用來詢問 LLM 的函數
def llm_ask(question, rag_contents):
    # 製作提問用的提詞（Prompt）
    formatted_prompt = f"Question: {question}\n\nContext: {rag_contents}"
    # 丟入 LLM，取得答案
    msg = llm.invoke(formatted_prompt)
    return msg

## 實際問答範例

In [ ]:
print(get_rag_docs("告訴我社會學相關課程的評價"))

**陳宗文**

課程名稱：社會學動動腦

學期：112-2

課程特色：教授有深度研究社會學的觀點，並分享個人經驗。期末展演是一組做一個海報。

**姜以琳**

課程名稱：社會學

學期：N/A

課程特色：適合入門，教授很可愛很有趣。考試有英文文獻。

**關秉寅**

課程名稱：社會學動動腦

學期：1071

課程特色：上課涼，給分極甜，但有學習價值。


In [ ]:
print(get_rag_docs_test("告訴我民法相關課程的評價"))

民法總則
參考資料：


**民法總則課程評量表**

| 教師 | 甜度 | 涼度 | 分數 | 推薦指數 |
|---|---|---|---|---|
| 吳瑾瑜 | 4/5 | 4/5 | 95 | 🍎🍎 |
| 牛曰正 | 4/5 | 3/5 | 80 | 🍎 |
| 周伯峰 | N/A | N/A | N/A | N/A |
| 牛曰正 | 4/5 | 3/5 | 90-94 | 🍎🍎 |
| 吳瑾瑜 | 4/5 | 3/5 | 90-94 | 🍎🍎 |

**評量標準：**

* **甜度：**課程教學風格、教材選定、教學互動等。
* **涼度：**課程內容的複雜程度、教學速度和質量。

**推薦指數：**

* **🍎🍎：**非常推薦。
* **🍎：**推薦。

**備註：**

* 教師的教學風格和教學品質會影響課程的甜度和涼度。
* 建議學生根據自己的學習需求和興趣選擇課程。


# 串接line bot (與個人綁定，無法在colab上運行)

## 安裝 ngrok 網頁伺服器相關套件

* 細節可以參考： https://pyngrok.readthedocs.io/en/latest/integrations.html#google-colaboratory

In [ ]:
# 安裝 pyngrok。一款專為 Python 打造的 ngrok 網頁伺服器。
!pip install pyngrok

In [ ]:
# 取得 ngrok 登入金鑰（Authtoken）並加入 Linux 系統中
# 登入 ngrok 主頁面： https://ngrok.com/
# 點擊左側欄 "Getting Started" > "Your Authtoken"
# 將畫面中央標示為 "Command Line" 的命令，拷貝貼上
# 金鑰（Authtoken）每人不同，請勿照抄別人的金鑰。
# 個人金鑰不會改變，請妥善保管。
!ngrok config add-authtoken 2pp658abCbmLJ9kpV49OOI9ERHV_4aqEEJpMmYD8hR73Zn17e

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## 攔截 LINE 訊息

* 細節請看這個網站： https://github.com/line/line-bot-sdk-python

In [ ]:
# 安裝 Line Bot SDK
!pip install line-bot-sdk

In [ ]:
# 攔截 LINE 訊息主程式，可由 LINE Bot SDK 網站拷貝得到

# 引入 Flask 網頁伺服器存取套件
from flask import Flask, request, abort

# 引入 Line Bot SDK 相關套件
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

# 引入 pyngrok 相關套件
import os
import threading
from pyngrok import ngrok



# 產生一個 Flask 物件，連上 Web 伺服器
app = Flask(__name__)

# 啟動 ngrok
# 將 ngrok 連往 Flask 預設埠號 5000
port = "5000"
# 取得 ngrok 公共 URL
public_url = ngrok.connect(port).public_url
# 印出 ngrok 公共 URL，並告知連往本地端埠號 5000
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")
# 將本服務的網址，由 localhost，轉為公共的 URL
app.config["BASE_URL"] = public_url

# Channel Acess Token 類似登入帳號，Channel Secret 類似登入密碼
# 兩者請至 LINE Developer 網站，相應 Channel 內取得
configuration = Configuration(access_token='qwcjeaCFzIGLARixUMmzln3sxZd/Q3K7rO+oJua8BjGct6/qpqAoHfc+l0AYdYyJzDm/Q+4wTD8Tf7qhWl+RnEJU4X1EQnM/Zbndi2JIaEu4bb1laKcZRYAtNhyZrEcwDMcGYtcmY27ymOHxOkeougdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('ab6299fe3bbe6e44cdeedd0db91c4647')

# 測試 ngrok 伺服器是否正常運作之函數
@app.route("/")
def index():
    return "Hello from ngrok!"

# 網頁封包處理函數，不要更動
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# 網頁訊息處理函數，可以客製化
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 使用 RAG 處理用戶輸入的訊息
        user_message = event.message.text
        try:
            reply_message = get_rag_docs(user_message)  # 使用 RAG 獲取回覆
        except Exception as e:
            # 若 RAG 出現錯誤，回傳預設訊息
            reply_message = "抱歉，我無法處理您的問題。"

        # 回覆用戶訊息
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_message)]
            )
        )

if __name__ == "__main__":
    # 以另一條執行緒將 Flask 啟動於背景，免得阻塞 Colab 使之無法操作
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


 * ngrok tunnel 'https://3f80-34-142-133-41.ngrok-free.app' -> 'http://127.0.0.1:5000'
 * Serving Flask app '__main__'


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


 * Debug mode: off
